<a href="https://colab.research.google.com/github/kapibara3chi/CooccurrenceAndWordCloud/blob/main/CooccurrenceNetworkAndWordCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリをインポート
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from wordcloud import WordCloud

# GInzaのモデルをロード
nlp = spacy.load("ja_ginza")

# 分析対象テキスト例として、Yahoo!ニュースのタイトルを用意[^3^][3]
text = """
首相「感染拡大防止に全力」　緊急事態宣言延長へ
東京五輪、海外観客受け入れ断念　政府、IOCと最終調整
「五輪中止も」首相発言に反響　与党内からも「選択肢に」
菅首相「大阪モデル」評価　医療提供体制の分析・改善要請
「コロナ禍でも安心」　東京・渋谷で自動運転バス、実証実験開始
"""

# テキストを文単位に分割し、形態素解析を行う
docs = [nlp(sent) for sent in text.split("\n") if sent]

# 名詞と動詞だけを抽出し、分かち書きしたリストを作成
words = []
for doc in docs:
    words.extend([token.lemma_ for token in doc if token.pos_ in ["NOUN", "VERB"]])

# Word Cloudを作成するための辞書を作成（単語と出現回数のペア）
word_dict = {}
for word in words:
    word_dict[word] = word_dict.get(word, 0) + 1

# Word Cloudのオブジェクトを作成
wc = WordCloud(width=800, height=600, background_color="white", font_path="C:\Windows\Fonts\meiryo.ttc")

# Word Cloudに辞書を渡して描画
wc.generate_from_frequencies(word_dict)
plt.figure(figsize=(10,8))
plt.imshow(wc)
plt.axis("off")
plt.show()

# 共起ネットワークを作成するための辞書を作成（単語のペアと共起回数のペア）
co_dict = {}
for doc in docs:
    tokens = [token.lemma_ for token in doc if token.pos_ in ["NOUN", "VERB"]]
    for i in range(len(tokens) - 1):
        for j in range(i + 1, len(tokens)):
            pair = tuple(sorted([tokens[i], tokens[j]]))
            co_dict[pair] = co_dict.get(pair, 0) + 1

# 共起ネットワークのオブジェクトを作成
G = nx.Graph()

# 共起回数が2以上の単語のペアだけをノードとエッジとして追加
for pair, freq in co_dict.items():
    if freq >= 2:
        G.add_node(pair[0])
        G.add_node(pair[1])
        G.add_edge(pair[0], pair[1], weight=freq)

# 共起ネットワークを描画
plt.figure(figsize=(10,10))
pos = nx.spring_layout(G, k=0.3) # ノードの位置をばねモデルで決定
nx.draw_networkx(G, pos, font_family="Meiryo", font_size=16,
                 node_color="skyblue", edge_color="gray", width=2) # ネットワークを描画
plt.axis("off")
plt.show()